In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
# setup keras/tensorflow
import os
import tensorflow as tf
# config for Gatsby cores
if 'gatsby' in os.environ['HOSTNAME']:

    # specify target gpu device
    os.environ['CUDA_VISIBLE_DEVICES'] = '1' # either '0' or '1' to utilize Titan X GPUs

    # allow dynamic GPU allocation
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    
import keras.backend as K
from keras import layers
from keras.models import Model
from keras.utils import to_categorical
from keras import objectives

In [ ]:
# Use much faster CuDNNLSTM if possible!
use_cudnn = True

LSTM = layers.CuDNNLSTM if use_cudnn else layers.LSTM

In [ ]:
# pick an input file to use for training:

#input_file = 'data/npy/mariel-1.npy'
input_file = 'data/npy/mariel-2.npy'
#input_file = 'data/npy/carrie-10-mins.npy'

# Do some simple formatting specific to each dataset.
# NB this notebook uses (time, vertex, dimension) axis
# ordering throughout.
# Uniform scaling is applied so that most sequences of moderate
# length can be contained within the range (-1,1) after translation;
# this may need to be adjusted for longer sequences.
def format_input(input_file):
    fname = os.path.basename(input_file)
    
    X0 = np.load(input_file)

    # depending on the input dataset, apply some simple
    # transformations and trim appropriate amounts from
    # the beginning/end of the file.
    if fname == 'mariel-1.npy':
        X0 = X0.transpose((1,0,2))
        X0 = X0[1300:7000]
        X0[:,:,2] *= -1
        X0 -= X0.mean(axis=(0,1))
        X0 *= 0.5
    elif fname == 'mariel-2.npy':
        X0 = X0.transpose((1,0,2))
        X0 = X0[1500:-1000]
        X0[:,:,2] *= -1
        X0 -= X0.mean(axis=(0,1))
        X0 *= 0.5
    elif fname == 'carrie-10-mins.npy':
        X0 = X0.transpose((1,0,2))
        X0 = X0[100:]
        X0[:,:,2] *= -1
        X0 -= X0.mean(axis=(0,1))
        X0 *= 0.5
    else:
        print("Warning! Don't know how to format file: %s" % input_file)
    
    return X0

X = format_input(input_file)

In [ ]:
# Set the fraction of data to use for validation, and partition
# the input data in time into train/validation sets.

validation_split = 0.15

# Split the input data into train and validate sets
n_train = int((1-validation_split)*X.shape[0])
n_val = X.shape[0]-n_train
X_train = X[:n_train]
X_val = X[n_train:]

In [ ]:
X[0].mean(axis=0).shape

In [ ]:
# define fancy plotting/animation functions

import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from mpl_toolkits.mplot3d.art3d import juggle_axes
from matplotlib import animation
from IPython.display import HTML

# ask matplotlib to plot up to 2^128 frames in animations
matplotlib.rcParams['animation.embed_limit'] = 2**128

def update_points(time, points, df):
  '''
  Callback function called by plotting function below. Mutates the vertex
  positions of each value in `points` so the animation moves
  @param int time: the index of the time slice to visualize within `df`
  @param mpl_toolkits.mplot3d.art3d.Path3DCollection points: the actual
    geometry collection whose internal values this function mutates to move
    the displayed points
  @param numpy.ndarray df: a numpy array with the following three axes:
    df.shape[0] = n_vertices
    df.shape[1] = n_time_slices
    df.shape[2] = n_dimensions
  '''
  points._offsets3d = juggle_axes(df[time,:,0], df[time,:,1], df[time,:,2], 'z')

def animate(seq, frames=None, axis_min=-0.75, axis_max=0.75, speed=45):
    if frames is None:
        frames = len(seq)
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.set_xlim(axis_min, axis_max)
    ax.set_ylim(axis_min, axis_max)
    ax.set_zlim(axis_min, axis_max)
    points = ax.scatter(seq[0,:,0], seq[0,:,1], seq[0,:,2], depthshade=False)
    return animation.FuncAnimation(
        fig,
        update_points,
        frames,
        interval=speed,
        fargs=(points, seq),
        blit=False,
    ).to_jshtml()

def plot_pose(x, ax=None, lim=(-0.5, 0.5), center=False, colors=None):
    if ax is None:
        fig = plt.figure()
        ax = p3.Axes3D(fig)
    if center:
        x = x-x.mean(axis=0)
    ax.set_xlim(*lim)
    ax.set_ylim(*lim)
    ax.set_zlim(*lim)
    ax.scatter(x[:,0], x[:,1], x[:,2], c=colors)
    return ax

def plot_stick(x, edges, ax=None, fig=None, subplot=None, lim=(-0.5,0.5), center=False, colors=None):
    if subplot is not None:
        if fig is None:
            fig = plt.figure()
        ax = fig.add_subplot(*subplot, projection='3d')
    if ax is None:
        if fig is None:
            fig = plt.figure()
        ax = p3.Axes3D(fig)
    if center:
        x = x-x.mean(axis=0)
    ax.set_xlim(*lim)
    ax.set_ylim(*lim)
    ax.set_zlim(*lim)
    #ax.scatter(x[:,0], x[:,1], x[:,2])
    for ie,e in enumerate(edges):
        if colors is not None:
            c = colors[ie]
        else:
            c = None
        ax.plot(np.linspace(x[e[0],0],x[e[1],0],10),np.linspace(x[e[0],1],x[e[1],1],10),np.linspace(x[e[0],2],x[e[1],2],10), color=c)
    return ax

In [ ]:
# calculate statistics about pairwise distances,
# for use in rigidity constraints

vdist_mean = np.zeros((X.shape[1],X.shape[1]))
vdist_var = np.zeros_like(vdist_mean)
vdist_norm = np.zeros_like(vdist_mean)
for i in range(X.shape[1]):
    for j in range(i+1,X.shape[1]):
        vdist = np.sum((X[:,i]-X[:,j])**2, axis=-1)
        vdist_mean[i,j] = vdist_mean[j,i] = vdist.mean()
        vdist_var[i,j] = vdist_var[j,i] = vdist.var(ddof=1)
        vdist_norm[i,j] = vdist_norm[j,i] = np.sqrt(vdist_var[i,j])/vdist_mean[i,j]
vdist_std = np.sqrt(vdist_var)

vdist_mean_q5, vdist_mean_q95 = np.quantile(vdist_mean[vdist_mean>0], (0.05,0.95))
vdist_var_q5, vdist_var_q95 = np.quantile(vdist_var[vdist_var>0], (0.05,0.95))
vdist_std_q5, vdist_std_q95 = np.quantile(vdist_std[vdist_std>0], (0.05,0.95))
vdist_norm_q5, vdist_norm_q95 = np.quantile(vdist_norm[vdist_norm>0], (0.05, 0.95))
print("pairwise mean quantiles:  5th=%.2e  95th=%.2e" % (vdist_mean_q5, vdist_mean_q95))
print("pairwise std. quantiles:  5th=%.2e  95th=%.2e" % (vdist_std_q5, vdist_std_q95))
plt.figure(figsize=(14,4))
plt.subplot(1,3,1)
plt.hist(np.log10(vdist_mean[vdist_mean>0]), bins=40);
plt.axvline(np.log10(vdist_mean_q5), color='red', lw=1);
plt.axvline(np.log10(vdist_mean_q95), color='red', lw=1);
plt.xlabel("log10(pairwise mean)", fontsize=14)
plt.subplot(1,3,2)
plt.hist(np.log10(vdist_std[vdist_std>0]), bins=40);
plt.xlabel("log10(pairwise std.)", fontsize=14);
plt.axvline(np.log10(vdist_std_q5), color='red', lw=1);
plt.axvline(np.log10(vdist_std_q95), color='red', lw=1);
plt.subplot(1,3,3)
plt.hist((vdist_norm[vdist_norm>0]), bins=40);
plt.axvline((vdist_norm_q5), color='red', lw=1);
plt.axvline((vdist_norm_q95), color='red', lw=1);
plt.xlabel("pairwise std/mean", fontsize=14);

In [ ]:
# plot matrices showing pairwise distance stats
plt.figure(figsize=(16,4))
plt.subplot(1,3,1)
plt.imshow(vdist_mean, vmin=vdist_mean_q5, vmax=vdist_mean_q95, norm=LogNorm())
plt.colorbar()
plt.title("pairwise mean", fontsize=14)
plt.subplot(1,3,2)
plt.imshow(vdist_std, vmin=vdist_std_q5, vmax=vdist_mean_q95, norm=LogNorm())
plt.title("pairwise std.", fontsize=14);
plt.colorbar();
plt.subplot(1,3,3)
plt.imshow(vdist_norm, vmin=vdist_norm_q5, vmax=vdist_norm_q95, norm=LogNorm())
plt.colorbar();
plt.title("pairwise std/mean");

In [ ]:
# sort the pairs by increasing pairwise std. dev.
upper_triangle = np.triu_indices_from(vdist_var, k=1)
variances = vdist_var[upper_triangle]
vtx_pairs = sorted(zip(*upper_triangle), key=lambda p: vdist_var[p[0],p[1]])

In [ ]:
# specify the number of rigid connections to use as constraints
# for drawing wireframe and/or imposing rigidity constraints
# the n_rigid lowest-variance pairs will be considered rigid
n_rigid = 150 # 140 seems to work well for mariel-1 and carrie-10; 150 works for mariel-2


plt.figure(figsize=(8,3))
plt.plot(sorted(vdist_var[upper_triangle]))
plt.yscale('log')
plt.ylabel('pairwise variance', fontsize=14)
plt.xlabel('pair #', fontsize=14)
plt.axvline(n_rigid, color='red', lw=1);
frame_num = 3500 #500
fig = plt.figure(figsize=(8,8))
ax = plot_stick(X[frame_num], vtx_pairs[:n_rigid], center=True, fig=fig)
plot_pose(X[frame_num], ax, center=True);
plt.title("Reference pose (frame %d)"%frame_num, fontsize=16);

In [ ]:
# Simple autoencoder to learn single-frame poses
# Parameters:
#   X: input dataset (used to determine shapes)
#   n_units: A tuple of 1 or more integers indicating the number of Dense layer units for each dense layer desired.
#   add_noise: if True, add unit gaussian noise to each of the encoded layer outputs.
#
# The returned model requires no target values during training/evaluation
# (since the target values are the same as the inputs).
def mk_pose_ae(X, n_units=(32,), add_noise=True):
    K.clear_session()
    
    # Define an activation for hidden layers
    def s(L):
        return layers.PReLU()(L)
    
    H = encoder_input = layers.Input((X.shape[1], X.shape[2]))
    
    # Shift the (x,y) values of the input so that it is centered at zero.
    # The z-coordinate is left unchanged.
    offsets = layers.Lambda(lambda x: K.constant([[[1,1,0]]])*K.mean(x,axis=1,keepdims=True))(encoder_input)
    H = layers.Subtract()([H, offsets])
    
    # Flatten vertices before feeding into dense networks.
    H = layers.Reshape((X.shape[1]*X.shape[2],))(H)
    
    # Create the specified number and size of Dense layers.
    for nu in n_units:
        H = layers.Dense(nu)(H)
        H = s(H)
    
    # add unit gaussian noise to the latent space, if requested.
    if add_noise:
        R = K.random_normal(K.shape(H), 0, 1)
        H = layers.Lambda(lambda x: x+R)(H)
    
    # Decoder layers progressively scale the latent space back to
    # original input size.
    for nu in n_units[::-1][1:]:
        H = layers.Dense(nu)(H)
        H = s(H)
    
    # Final dense output layer with tanh activation for (-1,1) range.
    H = layers.Dense((X.shape[1]*X.shape[2]), activation='tanh')(H)
    H = layers.Reshape((X.shape[1],X.shape[2]))(H)
    
    # restore the subtracted (x,y) offset before outputting
    H = layers.Add()([H, offsets])
    decoder_output = H
    
    autoencoder = Model(encoder_input, decoder_output)
    
    # Define the autoencoder loss as the pointwise mean squared error of the
    # output relative to the input.
    ae_loss = K.mean(K.sum(K.square(decoder_output-encoder_input), axis=-1))
    autoencoder.add_loss(ae_loss)
    
    # compile model without target y values (loss is defined by input and output layers only).
    autoencoder.compile(optimizer='adam')
    
    return autoencoder

In [ ]:
# Instantiate network with whatever parameters you like!

n_units = (64,)
add_noise = True

auto = mk_pose_ae(X, n_units=n_units, add_noise=add_noise)
auto.summary()

# these will get populated after each call to fit() by the values in the
# built-in History callback.
losses = []
val_losses = []

In [ ]:
# specify training parameters and fit the model.
# Note: subjectively, it seems that a loss of 2e-3 looks fairly convincing.
batch_size = 128
epochs = 16
learning_rate = 3e-4


K.set_value(auto.optimizer.lr, learning_rate)

auto.fit(X_train, None, validation_data=(X_val, None),
         epochs=epochs, batch_size=batch_size)

losses.extend(auto.history.history['loss'])
val_losses.extend(auto.history.history['val_loss'])

In [ ]:
# plot losses and some example auto-encoded poses from the validation set
plt.figure(figsize=(8,4))
plt.plot(losses, label='train')
plt.plot(val_losses, label='val')
plt.yscale('log')
plt.xlabel('epoch')
plt.ylabel('loss');

n_sample = 2
itest = np.random.randint(0,len(X_val),n_sample)
xtest = X_val[itest]
ytest = auto.predict(xtest)
for i in range(n_sample):
    fig = plt.figure(figsize=(10,6))
    plt.suptitle("Validation frame %d"% itest[i])
    plot_stick(xtest[i]-xtest[i].mean(axis=0), vtx_pairs[:n_rigid], center=True, fig=fig, subplot=(1,2,1))
    plot_stick(ytest[i]-ytest[i].mean(axis=0), vtx_pairs[:n_rigid], center=True, fig=fig, subplot=(1,2,2))

In [ ]:
# Now let's try to autoencode a whole sequence of poses.
def mk_seq_ae(X, seq_len=32, encoded_dim=32, lstm_units=(128,), add_noise=True):
    K.clear_session()
    
    noise_layer = layers.Lambda(lambda x: x+K.random_normal(K.shape(x)))
    
    
    autoencoder_input = H = layers.Input((seq_len, X.shape[1],X.shape[2],))
    
    # as before, we shift the entire sequence in (x,y) so that it's centered about (0,0)
    offsets = layers.Lambda(lambda x: K.constant([[[[1,1,0]]]])*K.mean(x,axis=(1,2),keepdims=True))(autoencoder_input)
    H = layers.Subtract()([H, offsets])
    H = layers.Reshape((seq_len, X.shape[1]*X.shape[2]))(H)
    
    # now encode the poses
    H = layers.Dense(encoded_dim)(H)
    H = layers.PReLU()(H)
    if add_noise:
        H = noise_layer(H)
    
    # branch off a secondary subnetwork that just immediately decodes
    # the sequence of decoded poses
    H2 = H
    H2 = layers.Dense(X.shape[1]*X.shape[2], activation='tanh')(H2)
    H2 = layers.Reshape((seq_len, X.shape[1],X.shape[2]))(H2)
    H2 = layers.Add()([H2, offsets])
    autoencoded_poses = H2
        
    # feed into LSTM(s)
    for nu in lstm_units:
        H = LSTM(nu, return_sequences=True)(H)
    
    # decode the poses in the LSTM sequence
    H = layers.Dense(X.shape[1]*X.shape[2], activation='tanh')(H)
    
    # un-shift the offsets subtracted in the beginning:
    H = layers.Reshape((seq_len, X.shape[1], X.shape[2]))(H)
    H = layers.Add()([H, offsets])
    autoencoder_output = H
    
    auto = Model(autoencoder_input, [autoencoder_output, autoencoded_poses])
    
    auto.hp_seq_weight = K.variable(1.0)
    auto.hp_pose_weight = K.variable(1.0)
    
    # define autoencoding loss as the vertex-wise mean squared error of the output vs. input
    ae_loss = K.mean(K.sum(K.square(autoencoder_input-autoencoder_output), axis=-1))
    auto.add_loss(auto.hp_seq_weight*ae_loss)
    
    pose_ae_loss = K.mean(K.sum(K.square(autoencoder_input-autoencoded_poses), axis=-1))
    auto.add_loss(auto.hp_pose_weight*pose_ae_loss)
    
    auto.compile(optimizer='adam')
    
    return auto

In [ ]:
# generator function to sample batches of contiguous sequences from a given dataset
def gen_batches(X, batch_size, seq_len):
    batch_idxs = np.arange(X.shape[0]-seq_len).repeat(seq_len).reshape(-1,seq_len) + np.arange(seq_len)
    nbatch = batch_idxs.shape[0]//batch_size
    while True:
        np.random.shuffle(batch_idxs)
        for ibatch in range(nbatch):
            #yield X[batch_idxs[ibatch*nbatch:(ibatch+1)*nbatch]], None
            yield X[batch_idxs[ibatch*batch_size:(ibatch+1)*batch_size]], None

In [ ]:
# specify training parameters and fit the model.
seq_len = 64
encoded_dim = 32
lstm_units = (16,)
add_noise = True

auto = mk_seq_ae(X, seq_len=seq_len, encoded_dim=encoded_dim, lstm_units=lstm_units, add_noise=add_noise)

losses = []
val_losses = []

In [ ]:
batch_size = 128
epochs = 8
learning_rate = 1e-3
seq_weight = 1.0
pose_weight = 0.0

K.set_value(auto.optimizer.lr, learning_rate)
K.set_value(auto.hp_seq_weight, seq_weight)
K.set_value(auto.hp_pose_weight, pose_weight)

nbatch_train = X_train.shape[0]//batch_size
nbatch_val = X_val.shape[0]//batch_size

auto.fit_generator(gen_batches(X_train, batch_size, seq_len),
                   validation_data=gen_batches(X_val, batch_size, seq_len),
                   validation_steps=nbatch_val,
                   steps_per_epoch=nbatch_train, epochs=epochs)

losses.extend(auto.history.history['loss'])
val_losses.extend(auto.history.history['val_loss'])

In [ ]:
plt.plot(losses[2:])
plt.plot(val_losses[2:])

In [ ]:
# pick a sequence from the validation set to visualize autoencoding performance
reference_sequence = X_val[640:640+seq_len]

seq_pred, pose_pred = auto.predict(np.expand_dims(reference_sequence,axis=0))
HTML(animate(seq_pred[0]))

In [ ]:
HTML(animate(pose_pred[0]))

In [ ]:
HTML(animate(reference_sequence))